In [2]:
import os
from nlpia.loaders import get_data

df = get_data('moviedialog')
df.columns = 'statement reply'.split()
df = df.dropna()
input_texts, target_texts = [], []  # <1>
start_token, stop_token = '\t\n'  # <3>
input_vocab = set()  # <2>
output_vocab = set(start_token + stop_token)
n_samples = min(100000, len(df))  # <4>

df['target'] = start_token + df.reply + stop_token 
[input_vocab.update(set(statement)) for statement in df.statement]
[output_vocab.update(set(reply)) for reply in df.reply]
input_vocab = tuple(sorted(input_vocab)) #<6>
output_vocab = tuple(sorted(output_vocab))

max_encoder_seq_len = df.statement.str.len().max()
# max_encoder_seq_len
# 100
max_decoder_seq_len = df.target.str.len().max()
# max_decoder_seq_len
# 102

In [3]:
import numpy as np  # <1> # noqa

encoder_input_onehot = np.zeros(
    (len(df), max_encoder_seq_len, len(input_vocab)),
    dtype='float32')  # <2>
decoder_input_onehot = np.zeros(
    (len(df), max_decoder_seq_len, len(output_vocab)),
    dtype='float32')
decoder_target_onehot = np.zeros(
    (len(df), max_decoder_seq_len, len(output_vocab)),
    dtype='float32')

for i, (input_text, target_text) in enumerate(
        zip(df.statement, df.target)):  # <3>
    for t, c in enumerate(input_text):  # <4>
        k = input_vocab.index(c)
        encoder_input_onehot[i, t, k] = 1.  # <5>
    k = np.array([output_vocab.index(c) for c in target_text])
    decoder_input_onehot[i, np.arange(len(target_text)), k] = 1.
    decoder_target_onehot[i, np.arange(len(target_text) - 1), k[1:]] = 1.

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

In [4]:
from keras.models import Model  # noqa
from keras.layers import Input, LSTM, Dense  # noqa

batch_size = 64    # <1>
epochs = 100       # <2>
num_neurons = 256  # <3>

encoder_inputs = Input(shape=(None, len(input_vocab)))
encoder = LSTM(num_neurons, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, len(output_vocab)))
decoder_lstm = LSTM(num_neurons, return_sequences=True,
                    return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(len(output_vocab), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.
2019-04-14 10:24:04,620 WARNING:     tensorflow:323:            new_func From /usr/local/miniconda3/envs/nlpiaenv/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


Instructions for updating:
Use tf.cast instead.
2019-04-14 10:24:05,096 WARNING:     tensorflow:323:            new_func From /usr/local/miniconda3/envs/nlpiaenv/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
2019-04-14 10:24:05,187 WARNING:     tensorflow:323:            new_func From /usr/local/miniconda3/envs/nlpiaenv/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:102: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 57915 samples, validate on 6435 samples
Epoch 1/100
57915/57915 [==============================] - 147s 3ms/step - loss: 0.7526 - acc: 0.1223 - val_loss: 0.6378 - val_acc: 0.15

57915/57915 [==============================] - 139s 2ms/step - loss: 0.3909 - acc: 0.2168 - val_loss: 0.4812 - val_acc: 0.2020
Epoch 51/100
57915/57915 [==============================] - 139s 2ms/step - loss: 0.3904 - acc: 0.2169 - val_loss: 0.4823 - val_acc: 0.2023
Epoch 52/100
57915/57915 [==============================] - 139s 2ms/step - loss: 0.3898 - acc: 0.2171 - val_loss: 0.4827 - val_acc: 0.2020
Epoch 53/100
57915/57915 [==============================] - 139s 2ms/step - loss: 0.3892 - acc: 0.2173 - val_loss: 0.4831 - val_acc: 0.2020
Epoch 54/100
57915/57915 [==============================] - 142s 2ms/step - loss: 0.3887 - acc: 0.2175 - val_loss: 0.4836 - val_acc: 0.2020
Epoch 55/100
57915/57915 [==============================] - 143s 2ms/step - loss: 0.3882 - acc: 0.2176 - val_loss: 0.4850 - val_acc: 0.2015
Epoch 56/100
57915/57915 [==============================] - 142s 2ms/step - loss: 0.3876 - acc: 0.2178 - val_loss: 0.4838 - val_acc: 0.2020
Epoch 57/100
57915/57915 [=======

In [ ]:
model.fit([encoder_input_onehot, decoder_input_onehot],
          decoder_target_onehot, batch_size=batch_size, epochs=epochs,
          validation_split=0.1)

In [5]:
from nlpia.loaders import get_data, DATA_PATH

In [6]:
model_path = os.path.join(DATA_PATH, 'ch10_train_movie_dialog_keras')
model.save(model_path + '_model.h5')

/usr/local/miniconda3/envs/nlpiaenv/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [7]:
model.save_weights(model_path + '_weights.h5')

In [8]:
from keras.models import load_model

model_path = os.path.join(DATA_PATH, 'ch10_train_movie_dialog_keras')
model = load_model(model_path + '_model.h5')

In [9]:
model.load_weights(model_path + '_weights.h5')

In [10]:
encoder_model = Model(encoder_inputs, encoder_states)
thought_input = [
    Input(shape=(num_neurons,)), Input(shape=(num_neurons,))]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=thought_input)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    inputs=[decoder_inputs] + thought_input,
    output=[decoder_outputs] + decoder_states)

/usr/local/miniconda3/envs/nlpiaenv/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  # Remove the CWD from sys.path while we load stuff.


In [13]:
def decode_sequence(input_seq):
    thought = encoder_model.predict(input_seq)  # <1>

    target_seq = np.zeros((1, 1, len(output_vocab)))  # <2>
    target_seq[0, 0, output_vocab.index(stop_token)
        ] = 1.  # <3>
    stop_condition = False
    generated_sequence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + thought) # <4>

        generated_token_idx = np.argmax(output_tokens[0, -1, :])
        generated_char = output_vocab[generated_token_idx]
        generated_sequence += generated_char
        if (generated_char == stop_token or
                len(generated_sequence) > max_decoder_seq_len
                ):  # <5>
            stop_condition = True

        target_seq = np.zeros((1, 1, len(output_vocab)))  # <6>
        target_seq[0, 0, generated_token_idx] = 1.
        thought = [h, c]  # <7>

    return generated_sequence

In [14]:
def respond(input_text):
    input_text = input_text.lower()
    input_text = ''.join(c if c in input_vocab else ' ' for c in input_text)
    input_seq = np.zeros((1, max_encoder_seq_len, len(input_vocab)), dtype='float32')
    for t, c in enumerate(input_text):
        input_seq[0, t, input_vocab.index(c)] = 1.
    decoded_sentence = decode_sequence(input_seq)
    print('Human: {}'.format(input_text))
    print('Bot:', decoded_sentence)
    return decoded_sentence

In [16]:
respond('Hi Rosa, how are you?')
respond('Hi Jim, how are you?')
respond('Hi Barak, how are you?')
respond('Hi Amy, how are you?')
respond('Hi Paris, how are you?')
respond('Hi Joe, how are you?')
respond('Hi Jane, how are you?')
respond('Hey Jane, how are you?')
respond('Hey Jon, how are you?')
respond('Hey John, how are you?')
respond('Hey Joe, how are you?')
respond('Hey Jim, how are you?')
respond('Hey Ashley, how are you?')
respond('Hey my love, how are you?')
respond('Hey Arzu, how are you?')
respond("I'm talking about us.")
respond("What are you trying to say?")

Human: hi rosa, how are you?
Bot: yes?

Human: hi jim, how are you?
Bot: hi.

Human: hi barak, how are you?
Bot: ...i was all right.

Human: hi amy, how are you?
Bot: hello, darling.

Human: hi paris, how are you?
Bot: everything i am.

Human: hi joe, how are you?
Bot: hi.

Human: hi jane, how are you?
Bot: hi.

Human: hey jane, how are you?
Bot: what are you going to do with this ship?

Human: hey jon, how are you?
Bot: hey, i was a little bit to be a man with you.

Human: hey john, how are you?
Bot: hey, what are you going to do to say?

Human: hey joe, how are you?
Bot: hey, i was a little bit to be a man with you.

Human: hey jim, how are you?
Bot: hey, man! you're gonna die what i say.

Human: hey ashley, how are you?
Bot: hey, what are you going to do to say?

Human: hey my love, how are you?
Bot: yes.

Human: hey arzu, how are you?
Bot: a start.

Human: i'm talking about us.
Bot: it's a long time ago. i think i should think i was a little late.

Human: what are you trying to say

'i was a company on the street.\n'